In [76]:
import gradio as gr
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
AI_token='****************************' 
MODEL = "gpt-4o-mini"

embeddings = OpenAIEmbeddings(base_url='https://api.gapgpt.app/v1', api_key=AI_token)

def create_vector_db(pdf_files_list):
    
    if not pdf_files_list:
        return None, gr.update(value="upload pdf file")

    all_chunks = []
    try:
        for pdf_file in pdf_files_list:
            loader = PyPDFLoader(pdf_file.name)
            documents = loader.load()
            text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=150)
            chunks = text_splitter.split_documents(documents=documents)
            all_chunks.extend(chunks)


        vector_store = Chroma.from_documents(documents=all_chunks, embedding=embeddings)
        
        return vector_store, gr.update(value="✅")

    except Exception as e:
        return None, gr.update(value=f"error: {e}")


def chat_response(message, chat_history, vector_store):
    if vector_store is None:
        chat_history.append((message, "First upload pdf file"))
        return "", chat_history

    llm = ChatOpenAI(base_url='https://api.gapgpt.app/v1', api_key=AI_token, temperature=0.7, model=MODEL)

    retriver=MultiQueryRetriever.from_llm(llm=llm,
                                         retriever=vector_store.as_retriever())

    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriver,
        memory=ConversationBufferMemory(memory_key='chat_history', return_messages=True)
    )

    result = conversation_chain.invoke({"question": message})

    chat_history.append((message, result['answer']))

    return "", chat_history


with gr.Blocks(theme=gr.themes.Citrus()) as ui:
    gr.Markdown("<h1 style='text-align: center;'>🤖 RAG Chatbot 🤖</h1>")
    
  
    vector_store_state = gr.State(None)

    with gr.Row():
        with gr.Column(scale=1):
            file_input = gr.File(
                label='Upload your pdf file',
                file_types=['.pdf'],
                file_count='multiple',
                height=300
            )
            create_button = gr.Button('Build vector', variant="primary")
            status_output = gr.Textbox(label="status", interactive=False)

        with gr.Column(scale=2):
            chatbot_output = gr.Chatbot(height=350)
            entry_input = gr.Textbox(label="Press enter")


    create_button.click(
        fn=create_vector_db,
        inputs=[file_input],
        outputs=[vector_store_state, status_output]
    )

  
    entry_input.submit(
        fn=chat_response,
        inputs=[entry_input, chatbot_output, vector_store_state],
        outputs=[entry_input, chatbot_output]
    )

ui.launch()

C:\Users\ParsRayaneh\AppData\Local\Temp\ipykernel_12700\27494847.py:78: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot_output = gr.Chatbot(height=350)


* Running on local URL:  http://127.0.0.1:7908
* To create a public link, set `share=True` in `launch()`.
